Realtime adjust tries to rerank the categories based on customers' realtime actions, we initially only consider sales action. <p>
The assumptions here are: <br>
- If a certain customer purchased one category, this category should be demoted by a factor, and all categories will be ranked again. because this customer highly likely will not puchase this category again within a samll time window. <br>
- We also assume if we give more impression chances for the low ranking categories to show more diverse categories, there is a long-term gain of the whole platform. There is chance that model woll choose one of long-tailed categories as the topmost. <br>
    
We add these additonal metrics to evaluate the performance of the realtime rerankings.
It includes impressions, clicks, sales changes at category level, such as total clicks, sales, and click ratio and sale ratio. <br>
The distribution should become evener(exploration)


In [1]:
%%configure -f
{
    "driverMemory":"4G",
    "executorMemory":"20G",
    "executorCores":4,
    "numExecutors":10
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
28,application_1623160956156_0149,pyspark,idle,Link,Link,
29,application_1623160956156_0153,pyspark,idle,Link,Link,


In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
30,application_1623160956156_0156,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#sc.install_pypi_package('scipy')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
date="2021-06-02"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
outpath='s3a://pai-gm-midgar-shared-ml-team/pengcheng/rt/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [157]:
raw_index="temp_ranked_l1_trendfiltered_scores_raw"
rt_index="temp_ranked_l1_trendfiltered_scores_rt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# load customers affected by rt

In [158]:
customer_df=spark.read.parquet(outpath+f"/parsed_logs/dt={date}").select('customer_id').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
customer_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12048

# model info

In [160]:
cols=['l1_category_cma_id', 'l1_category_name', 'banner_id', 'customer_id', 'date', 'experiment_group_name']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
p1="s3a://midgar-aws-workspace/prod/telescope/dashboard/ab_testing/banner_events_adjust/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
model_df=spark.read.parquet(p1+f"/dt={date}").select(cols).join(customer_df, on=['customer_id'])  ## only consider affected customers

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [163]:
model_df.show(2,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+----------------+---------+----------+----------------------------------------------------+
|customer_id|l1_category_cma_id|l1_category_name|banner_id|date      |experiment_group_name                               |
+-----------+------------------+----------------+---------+----------+----------------------------------------------------+
|10585129   |6801              |CL-PP           |729305   |2021-06-02|temp_ranked_l1_trendfiltered_ids_banner_events_sales|
|133397418  |18                |EC-HnK          |821527   |2021-06-02|temp_ranked_l1_trendfiltered_ids_banner_events_sales|
+-----------+------------------+----------------+---------+----------+----------------------------------------------------+
only showing top 2 rows

In [164]:
model_df.select('experiment_group_name').distinct().show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------+
|experiment_group_name                               |
+----------------------------------------------------+
|temp_ranked_l1_trendfiltered_scores_raw             |
|temp_extreme_affinity_dampening                     |
|temp_ranked_l1_trendfiltered_scores_rt              |
|temp_ranked_l1_trendfiltered_ids_banner_events_sales|
+----------------------------------------------------+

# valid categories

In [165]:
valid_categories=spark.read.option("header","true").csv(outpath+f"/valid_categories/dt={date}").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
valid_categories.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25

In [167]:
valid_categories.show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|sale_l1_category_id|
+-------------------+
|443                |
+-------------------+
only showing top 1 row

In [168]:
#model_df=model_df.join(valid_categories, model_df.l1_category_cma_id==valid_categories.sale_l1_category_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# ctr, cvr

In [169]:
bpath="s3a://midgar-aws-workspace/prod/shinra/telescope/dashboard/banner_campaigns/customer_level/banner_campaign_base_customer_level_full/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
cols=['customer_id', 'banner_id', 'l1_category_name', 'date', 'impressions_adjust', 'clicks_adjust', 'category_sales_adjust', 'category_gmv_adjust']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
df=spark.read.parquet(bpath+f"/dt={date}").select(cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- banner_id: string (nullable = true)
 |-- l1_category_name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- impressions_adjust: double (nullable = true)
 |-- clicks_adjust: double (nullable = true)
 |-- category_sales_adjust: double (nullable = true)
 |-- category_gmv_adjust: double (nullable = true)

In [173]:
df=df.join(model_df, on=['customer_id', 'banner_id', 'date', 'l1_category_name'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [174]:
df.show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------+----------------+------------------+-------------+---------------------+-------------------+------------------+----------------------------------------------------+
|customer_id|banner_id|date      |l1_category_name|impressions_adjust|clicks_adjust|category_sales_adjust|category_gmv_adjust|l1_category_cma_id|experiment_group_name                               |
+-----------+---------+----------+----------------+------------------+-------------+---------------------+-------------------+------------------+----------------------------------------------------+
|1234366207 |805675   |2021-06-02|RU-Prepaid      |1.0               |0.0          |0.0                  |0.0                |443               |temp_ranked_l1_trendfiltered_ids_banner_events_sales|
+-----------+---------+----------+----------------+------------------+-------------+---------------------+-------------------+------------------+----------------------------------------------------+
only 

In [175]:
## save customers

#(df.select('customer_id').distinct().write.mode('overwrite').parquet(outpath+f"/banner_customers/dt={date}"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [176]:
df2=df.groupBy('l1_category_name', 'l1_category_cma_id', 'date', 'experiment_group_name')\
.agg(F.sum('impressions_adjust').alias('impressions_total'),
     F.sum('clicks_adjust').alias('clicks_total'),
     F.sum('category_sales_adjust').alias('category_sales_total'),
     F.sum('category_gmv_adjust').alias('category_gmv_total'))\
.withColumn('ctr', F.col('clicks_total')/F.col('impressions_total'))\
.withColumn('cvr', F.col('category_sales_total')/F.col('impressions_total'))\
.withColumn('gmv_per_impression', F.col('category_gmv_total')/F.col('impressions_total'))\
.withColumn('gmv_per_click', F.col('category_gmv_total')/F.col('clicks_total'))\
.sort(F.col('l1_category_cma_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
(df2.coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/ctr_cvr/dt={date}"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df3=spark.read.option("header","true").csv(outpath+f"/ctr_cvr/dt={date}").drop('date')
df3=df3.join(valid_categories, df3.l1_category_cma_id==valid_categories.sale_l1_category_id, 'left') ## valid categories

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [179]:
df3.select('experiment_group_name').distinct().show(10, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------------------+
|experiment_group_name                               |
+----------------------------------------------------+
|temp_ranked_l1_trendfiltered_scores_raw             |
|temp_extreme_affinity_dampening                     |
|temp_ranked_l1_trendfiltered_scores_rt              |
|temp_ranked_l1_trendfiltered_ids_banner_events_sales|
+----------------------------------------------------+

In [180]:
raw_df=df3.filter(F.col('experiment_group_name')==raw_index).drop('experiment_group_name')
raw_df=raw_df.withColumn('impression_ratio', F.col('impressions_total')/F.sum(F.col('impressions_total')).over(Window.partitionBy()))\
    .withColumn('clicks_ratio', F.col('clicks_total')/F.sum(F.col('clicks_total')).over(Window.partitionBy()))
raw_df=raw_df.toDF(*(c+'_raw' for c in raw_df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
raw_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['l1_category_name_raw', 'l1_category_cma_id_raw', 'impressions_total_raw', 'clicks_total_raw', 'category_sales_total_raw', 'category_gmv_total_raw', 'ctr_raw', 'cvr_raw', 'gmv_per_impression_raw', 'gmv_per_click_raw', 'sale_l1_category_id_raw', 'impression_ratio_raw', 'clicks_ratio_raw']

In [182]:
rt_df=df3.filter(F.col('experiment_group_name')==rt_index).drop('experiment_group_name', 'l1_category_cma_id')
rt_df=rt_df.withColumn('impression_ratio', F.col('impressions_total')/F.sum(F.col('impressions_total')).over(Window.partitionBy()))\
    .withColumn('clicks_ratio', F.col('clicks_total')/F.sum(F.col('clicks_total')).over(Window.partitionBy()))
rt_df=rt_df.toDF(*(c+'_rt' for c in rt_df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [183]:
rt_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['l1_category_name_rt', 'impressions_total_rt', 'clicks_total_rt', 'category_sales_total_rt', 'category_gmv_total_rt', 'ctr_rt', 'cvr_rt', 'gmv_per_impression_rt', 'gmv_per_click_rt', 'sale_l1_category_id_rt', 'impression_ratio_rt', 'clicks_ratio_rt']

In [184]:
agg_df=raw_df.join(rt_df, raw_df.l1_category_name_raw==rt_df.l1_category_name_rt, "outer")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [185]:
agg_df=agg_df.withColumn('impressions_total_raw', F.col('impressions_total_raw').cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
cols=['l1_category_name_raw', 'l1_category_cma_id_raw', 'sale_l1_category_id_raw', 'sale_l1_category_id_rt',
      'l1_category_name_rt',
      'impressions_total_raw', 'impressions_total_rt',
 'clicks_total_raw', 'clicks_total_rt',
 'category_sales_total_raw', 'category_sales_total_rt', 
 'category_gmv_total_raw', 'category_gmv_total_rt', 
 'ctr_raw', 'ctr_rt',
 'cvr_raw', 'cvr_rt',
 'gmv_per_impression_raw', 'gmv_per_impression_rt',
 'gmv_per_click_raw', 'gmv_per_click_rt',
 'impression_ratio_raw', 'impression_ratio_rt',
 'clicks_ratio_raw', 'clicks_ratio_rt']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [187]:
(agg_df.select(cols).sort(F.col('impressions_total_raw').desc()).coalesce(1)
.write.mode('overwrite')
.option("header","true")
.csv(outpath+f"/ctr_cvr_agg/dt={date}"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# entropy of impression ratio

In [8]:
def get_entropy(exp, col_name, date):
    import math
    rdf=spark.read.option("header","true").csv(outpath+f"/ctr_cvr/dt={date}").filter(F.col('experiment_group_name')==exp)
    #rdf=rdf.join(categories, rdf.l1_category_cma_id==categories.sale_l1_category_id, ) ## valid categories
    rdf=rdf.withColumn('impression_ratio', F.col('impressions_total')/F.sum(F.col('impressions_total')).over(Window.partitionBy()))\
    .withColumn('clicks_ratio', F.col('clicks_total')/F.sum(F.col('clicks_total')).over(Window.partitionBy()))
    rdf=rdf.select(col_name)
    print(rdf.count())
    rdf.show(20,False)
    vs=[row[0] for row in rdf.select(col_name).collect()]
    vs=list(map(float, vs))
    return sum([-i * math.log(i) for i in vs])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
get_entropy(raw_index, 'impression_ratio', '2021-06-01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

51
+---------------------+
|impression_ratio     |
+---------------------+
|0.01623511581085401  |
|0.019982335971068767 |
|0.02258317458671049  |
|0.017374530823420862 |
|0.017162438626788172 |
|0.017077292124490377 |
|0.020120118492968835 |
|0.017330409454048366 |
|0.01864708400321699  |
|0.018192711304591488 |
|0.01769421723659349  |
|0.036023937003973244 |
|0.0172746771979989   |
|0.00582092452072195  |
|4.7991664931484156E-5|
|0.021144198697877762 |
|0.006190924776161457 |
|0.016406956933673198 |
|0.001711444696185669 |
|0.019822105734926553 |
+---------------------+
only showing top 20 rows

3.7541446253370703

In [16]:
get_entropy(rt_index, 'impression_ratio', '2021-06-01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

51
+---------------------+
|impression_ratio     |
+---------------------+
|0.01625307965414196  |
|0.020394414311453345 |
|0.019921341964171405 |
|0.01749352843274098  |
|0.017340521534940218 |
|0.017144579028062715 |
|0.024519355372571796 |
|0.017679322522927615 |
|0.01882219037180676  |
|0.01818908529978423  |
|0.018214846665230277 |
|0.03677161690819898  |
|0.017646535330541738 |
|0.005832217007497338 |
|4.371625651450287E-5 |
|0.020977557947459306 |
|0.006261573098264777 |
|0.017182830752512906 |
|0.0016167208435988471|
|0.020028290663144384 |
+---------------------+
only showing top 20 rows

3.7626334347168444